In [2]:
import pandas as pd
import os
import gzip
import math
import numpy as np
import scipy.stats as st
import sys
sys.path.append(os.path.abspath(os.path.join('..')))
import common

In [56]:
def get_merge_dict(merge_info_path, tool_name):
    merge_dict = dict()
    df = pd.read_csv(merge_info_path)
    df = df[df["program"] == tool_name].reset_index()
    for index, row in df.iterrows():
        key = (row["type"], row["qs1"], row["qs2"])
        merge_dict[key] = (int(row["new_nt"][1]), row["new_qs"])
    return merge_dict


def _next_fastq_entry(fastq_file, fraglen):
    entry = dict()
    entry["name"] = fastq_file.readline().rstrip()[1:-4]
    entry["sequence"] = fastq_file.readline().rstrip()[:fraglen]
    entry["optional"] = fastq_file.readline().rstrip()
    entry["quality"] = fastq_file.readline().rstrip()[:fraglen]
    return entry


def _next_fasta_entry(fasta_file):
    entry = dict()
    entry["name"] = fasta_file.readline().rstrip()[1:]
    entry["sequence"] = fasta_file.readline().rstrip()
    return entry


def get_next_entries(fragments_file, s1_file, s2_file, fraglen, dna_complement):
    """
    Returns the next fasta entry of the file with the original 
    fragments, and the next fastq entries of the simulated reads. For 
    the mate2 read, the reverse complement sequence and the reverse 
    quality score line will be returned. Furthermore, the function will 
    raise an Assertion error, if the names of all sequences don't align.
    """
    orig = _next_fasta_entry(fragments_file)
    read1 = _next_fastq_entry(s1_file, fraglen)
    read2 = _next_fastq_entry(s2_file, fraglen)
    read2["sequence"] = read2["sequence"].translate(dna_complement)[::-1]
    read2["quality"] = read2["quality"][::-1]
    assert orig["name"] == read1["name"]
    assert orig["name"] == read2["name"]
    return orig, read1, read2


def merged_base_and_qual(merge_dict, base1, base2, qual1, qual2):
    # find out if the base matches in read1 and read2
    if base1 == base2:
        match_type = "match"
    else:
        match_type = "mismatch"
    # get the merged base and quality score
    new_nt, merged_qual = merge_dict[(match_type, qual1, qual2)]
    merged_base = (base1, base2)[new_nt-1]
    return merged_base, merged_qual


def add_to_phred_counter(phred_counter, merged_qual, merged_base, orig_base):
    # add to total count
    try:
        phred_counter[merged_qual]["total_cnt"] += 1
    except KeyError:
        # phred score not in dict, create key/value pair
        phred_counter[merged_qual] = {
            "total_cnt": 1, 
            "error_cnt": 0
            }
    # add to counter
    if merged_base != orig_base:
        phred_counter[merged_qual]["error_cnt"] += 1


def process_reads(orig, read1, read2, merge_dict, phred_counter):
    for pos in range(len(orig["sequence"])):
        # get merged base and quality
        merged_base, merged_qual = merged_base_and_qual(
            merge_dict, 
            read1["sequence"][pos], 
            read2["sequence"][pos], 
            read1["quality"][pos] - 33, 
            read2["quality"][pos] - 33,
            )
        # add count for the quality score
        add_to_phred_counter(
            phred_counter,
            merged_qual,
            merged_base,
            orig["sequence"][pos]
            )


def phred_2_p_error(q_value):
    p_error = 10**(-int(q_value)/10)
    return p_error


def p_mismatch(n_total, n_mismatch):
    if n_total == 0:
        p = np.NAN
    else:
        p = n_mismatch/n_total
    return p


def binomial_ci(n, k, alpha):
    """ 
    Exact Confidence Interval
    https://sigmazone.com/binomial-confidence-intervals/ 
    """
    if k == 0:
        p_lower = 0
    else: 
        p_lower = 1 - st.beta.ppf(1-(alpha/2), n-k+1 , k)
    if k == n:
        p_upper = 1
    else:
        p_upper = 1 - st.beta.ppf(alpha/2, n-k , k+1)
    return p_lower, p_upper


def p_error_2_phred(p_err, max_phred=100):
    # probability of incorrect base call
    if p_err <= 0:
        phred_observed = max_phred
    elif p_err == 1:
        phred_observed = 0
    else: 
        phred_observed = -10 * math.log10(p_err)
    return phred_observed


def get_results(phred_counter, alpha):
    # set a maximum value for the observed phred, this is needed if the
    # observed error rate is 0
    max_phred = 100
    
    results = {'predicted_phred': list(),
        'predicted_error': list(),
        'n_matches': list(),
        'n_mismatches': list(),
        'n_total': list(),
        'p_mismatch': list(),
        'p_mismatch_lower': list(),
        'p_mismatch_upper': list(),
        'observed_phred': list(),
        'observed_phred_lower': list(),
        'observed_phred_upper': list(),
        }
    
    for q_score in sorted(phred_counter.keys()):
        
        predicted_error = phred_2_p_error(q_score)
        n_matches = phred_counter[q_score]["match_cnt"]
        n_mismatches = phred_counter[q_score]["mismatch_cnt"]
        n_total = n_mismatches + n_matches
        p_mm = p_mismatch(n_total, n_mismatches)
        p_mm_lower, p_mm_upper = binomial_ci(n_total, n_mismatches, alpha)
        observed_phred = p_error_2_phred(p_mm, max_phred)
        observed_phred_lower = p_error_2_phred(p_mm_upper, max_phred)
        observed_phred_upper = p_error_2_phred(p_mm_lower, max_phred)
    
        results["predicted_phred"].append(q_score)
        results["predicted_error"].append(predicted_error)
        results["n_matches"].append(n_matches)
        results["n_mismatches"].append(n_mismatches)
        results["n_total"].append(n_total)
        results["p_mismatch"].append(p_mm)
        results["p_mismatch_lower"].append(p_mm_lower)
        results["p_mismatch_upper"].append(p_mm_upper)
        results["observed_phred"].append(observed_phred)
        results["observed_phred_lower"].append(observed_phred_lower)
        results["observed_phred_upper"].append(observed_phred_upper)
    
    return results


def main(template_path, s1_path, s2_path, merge_info_path, nfrags, fraglen, qualityshift, tool_name,
         export_path=None):

    alpha = 0.01

    # Load files
    templates_file = gzip.open(template_path, "rb") if common._is_gzipped(template_path) else open(template_path, "rb")
    s1_file = gzip.open(s1_path, "rb") if common._is_gzipped(s1_path) else open(s1_path, "rb")
    s2_file = gzip.open(s2_path, "rb") if common._is_gzipped(s2_path) else open(s2_path, "rb")
    merge_dict = get_merge_dict(merge_info_path, tool_name)

    # Analysis
    phred_counter = dict()
    dna_complement = bytes.maketrans(b"ACTG", b"TGAC")
    for _ in range(nfrags):
        orig, read1, read2 = get_next_entries(templates_file, s1_file, s2_file, fraglen, dna_complement)
        process_reads(orig, read1, read2, merge_dict, phred_counter)
    
    results = get_results(phred_counter, alpha)


    # Export results 
    if export_path is not None:
        df = pd.DataFrame.from_dict(results)
        df.insert(0, 'program', tool_name)
        df.insert(1, 'nfrags', nfrags)
        df.insert(2, 'fraglen', fraglen)
        df.insert(3, 'qual_shift', qualityshift)
        df.insert(4, 'alpha', alpha)
        df.to_csv(export_path, na_rep="NA")
    

In [36]:
template_path = os.path.join("output", "simulations", "gen_n1000000_l120_frag.fa")
s1_path = os.path.join("output", "simulations", "gen_n1000000_l120_s1.fq")
s2_path = os.path.join("output", "simulations", "gen_n1000000_l120_s2.fq")
merge_info_path = "input/all_qs.csv"
tool_name = "fastp"
fraglen = 120
nfrags = 1000000
alpha = 0.01
export_path = None
qualityshift = 0

In [38]:
# Load files
templates_file = gzip.open(template_path, "rb") if common._is_gzipped(template_path) else open(template_path, "rb")
s1_file = gzip.open(s1_path, "rb") if common._is_gzipped(s1_path) else open(s1_path, "rb")
s2_file = gzip.open(s2_path, "rb") if common._is_gzipped(s2_path) else open(s2_path, "rb")
merge_dict = get_merge_dict(merge_info_path, tool_name)

In [67]:
# Analysis
phred_counter2 = dict()
dna_complement = bytes.maketrans(b"ACTG", b"TGAC")
for _ in range(nfrags):
    orig, read1, read2 = get_next_entries(templates_file, s1_file, s2_file, fraglen, dna_complement)
    break 

for pos in range(len(orig["sequence"])):
    orig_base = orig["sequence"][pos]
    base1 = read1["sequence"][pos]
    base2 = read2["sequence"][pos]
    # get the quality score of read1 and read2
    qual1 = read1["quality"][pos] - 33
    qual2 = read2["quality"][pos] - 33

    merged_base, merged_qual = merged_base_and_qual(merge_dict, 
                                                        base1, base2, 
                                                        qual1, qual2)
    # add count for the quality score
    add_to_phred_counter(phred_counter2, merged_qual, merged_base, orig_base)
    break

In [85]:
phred_counter = dict()
phred_counter2

{33: {'total_cnt': 1, 'error_cnt': 0}}

In [83]:
def merge_singlethread_counter(phred_counter_list, phred_counter):
    for phred_counter_st in phred_counter_list:
        for phred in phred_counter_st.keys():
            try:
                phred_counter[phred]["total_cnt"] \
                    += phred_counter_st[phred]["total_cnt"]
                phred_counter[phred]["error_cnt"] \
                    += phred_counter_st[phred]["error_cnt"]
            except KeyError:
                phred_counter[phred] = phred_counter_st[phred]

In [87]:
merge_singlethread_counter([phred_counter2], phred_counter)
phred_counter

{33: {'total_cnt': 2, 'error_cnt': 0}}